<a href="https://colab.research.google.com/github/hafeezjaan77/AMP_Data/blob/main/Replicase_polyprotein_1b_Part_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Replicase polyprotein 1b Bioactivity Data**
Abdul Hafeez


# **ChEMBL Database**


**Installing libraries**

Install the ChEMBL web service package so that we can retrieve bioactivity data from the ChEMBL Database.

In [ ]:
! pip install chembl_webresource_client

# **Importing libraries**

In [ ]:
# Import necessary libraries
import pandas as pd
from chembl_webresource_client.new_client import new_client

# **Search for Target protein**

**Target search for Nisin**

In [ ]:
# Target search for 
target = new_client.target
target_query = target.search('antimicrobial peptide')
targets = pd.DataFrame.from_dict(target_query)
targets

**Select and retrieve bioactivity data for Replicase polyprotein 1ab (second entry)**

We will assign the second entry (which corresponds to the target protein, Replicase polyprotein 1ab) to the selected_target variable

In [ ]:
selected_target = targets.target_chembl_id[172]
selected_target

Here, we will retrieve only bioactivity data for Replicase polyprotein 1b (CHEMBL4523582) that are reported as pChEMBL values.



In [ ]:
activity = new_client.activity
res = activity.filter(target_chembl_id=selected_target).filter(standard_type="IC50")

In [ ]:
df = pd.DataFrame.from_dict(res)

In [ ]:
df

Finally we will save the resulting bioactivity data to a CSV file bioactivity_data.csv.



In [ ]:
df.to_csv('replicase polyprotein 1b_01_bioactivity_data_raw.csv', index=False)

# **Handling missing data**

If any compounds has missing value for the standard_value and canonical_smiles column then drop it.

In [ ]:
df2 = df[df.standard_value.notna()]
df2 = df2[df.canonical_smiles.notna()]
df2

In [ ]:
len(df2.canonical_smiles.unique())


In [ ]:
df2_nr = df2.drop_duplicates(['canonical_smiles'])
df2_nr

# **Data pre-processing of the bioactivity data**

Combine the 3 columns (molecule_chembl_id,canonical_smiles,standard_value) and bioactivity_class into a DataFrame

In [ ]:
selection = ['molecule_chembl_id','canonical_smiles','standard_value']
df3 = df2_nr[selection]
df3

Saves dataframe to CSV file



In [ ]:
df3.to_csv('replicase polyprotein 1b_02_bioactivity_data_preprocessed.csv', index=False)


# **Labeling compounds as either being active, inactive or intermediate**

The bioactivity data is in the IC50 unit. Compounds having values of less than 1000 nM will be considered to be active while those greater than 10,000 nM will be considered to be inactive. As for those values in between 1,000 and 10,000 nM will be referred to as intermediate.

In [ ]:
df4 = pd.read_csv('replicase polyprotein 1b_02_bioactivity_data_preprocessed.csv')

In [ ]:
bioactivity_threshold = []
for i in df4.standard_value:
  if float(i) >= 10000:
    bioactivity_threshold.append("inactive")
  elif float(i) <= 1000:
    bioactivity_threshold.append("active")
  else:
    bioactivity_threshold.append("intermediate")

In [ ]:
bioactivity_class = pd.Series(bioactivity_threshold, name='class')
df5 = pd.concat([df4, bioactivity_class], axis=1)
df5

Saves dataframe to CSV file



In [ ]:
df5.to_csv('replicase popyprotein 1b_03_bioactivity_data_curated.csv', index=False)


In [ ]:
! zip replicase polyprotein 1b.zip *.csv


In [ ]:
! ls -l